## Respondendo perguntas de negócio usando SQL

**Brazilian E-Commerce Public Dataset by Olist**

[https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce)

Olist é uma empresa brasileira que ajuda as pessoas a vender os produtos em diversos e-commerce. Ela cadastra o produto em varios marketing places em diferentes plataformas. Você cadastra uma vez no site da Olist e ela faz o trabalho de cadastrar o seu produto em todos as plataforma que vocês quiser.

**Schema do dataset**

<img src="schema_olist.png" >

**Perguntas feita pelo time de negócio:**

01. Quantos pedidos foram feitos para cada tipo de pagamento?
02. Qual o número máximo e mínimo de parcelas nos pagamentos?
03. Quais são os top 10 pedidos com os maiores valores?
04. Quais são os últimos 10 pedidos com os menores valores?
05. Qual a média do valor de pagamento por tipo de pagamento?
06. Quais os top 5 clientes com os maiores valores de pagamento no boleto?
07. Quais os top 5 clientes com os maiores valores de pagamento no cartão de crédito?
08. Quais os 10 produtos mais caros?
09. Quais os 10 produtos mais baratos?
10. Quais as 10 categorias mais compradas?
11. Quais os 5 produtos com maior número de reviews?
12. Quais os top 10 produtos sem nenhum review?
13. Quais os 10 clientes com maior quantidade de pedidos?
14. Quais os 10 clientes com a menor quantidade de pedidos?
15. Quais vendedores existem na base?
16. Qual a distribuição de vendedores por estado?
17. Qual a distribuição de clientes por estado?
18. Quais são os top 10 vendedores que mais receberam pagamentos por boleto?
19. Quais são os 10 piores vendedores em termos de número de vendas?
20. Quantos produtos são comprados, em média, por pedido?

In [1]:
#carregando as bibliotecas
import sqlite3
from sqlalchemy import create_engine
import pandas as pd

In [2]:
#conectando com o database
db = create_engine('sqlite:///db_olist.sqlite', echo=False)
conn = db.connect()

In [3]:
query = """
SELECT name FROM sqlite_master WHERE type='table'
"""

table = pd.read_sql_query(query,conn)
table

,name
0,customer
1,geolocation
2,items
3,order_payments
4,order_reviews
5,orders
6,products
7,sellers
8,product_category_name


### 01. Quantos pedidos foram feitos para cada tipo de pagamento?

In [4]:
query = """
SELECT * FROM order_payments
"""

table = pd.read_sql_query(query,conn)
table

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45
...,...,...,...,...,...
103881,0406037ad97740d563a178ecc7a2075c,1,boleto,1,363.31
103882,7b905861d7c825891d6347454ea7863f,1,credit_card,2,96.80
103883,32609bbb3dd69b3c066a6860554a77bf,1,credit_card,1,47.77
103884,b8b61059626efa996a60be9bb9320e10,1,credit_card,5,369.54


In [10]:
query1 = """
select payment_type,
    count(payment_type) as qtd_pedidos
from order_payments
group by payment_type

"""

df1 = pd.read_sql_query(query1, con=conn)
df1

,payment_type,qtd_pedidos
0,boleto,19784
1,credit_card,76795
2,debit_card,1529
3,not_defined,3
4,voucher,5775


### 02. Qual o número máximo e mínimo de parcelas nos pagamentos?

In [15]:
query2 = """
select max(payment_installments) as max_parcela,
       min(payment_installments) as min_parcela
from order_payments
"""

df2 = pd.read_sql_query(query2, con=conn)
df2

,max_parcela,min_parcela
0,24,0


### 03. Quais são os top 10 pedidos com os maiores valores?

In [53]:
query3 = """
select order_id,
       payment_value
from order_payments
order by payment_value desc
limit 10
"""

df3 = pd.read_sql_query(query3, con=conn)
df3

,order_id,payment_value
0,03caa2c082116e1d31e67e9ae3700499,13664.08
1,736e1922ae60d0d6a89247b851902527,7274.88
2,0812eb902a67711a1cb742b3cdaa65ae,6929.31
3,fefacc66af859508bf1a7934eab1e97f,6922.21
4,f5136e38d1a14a4dbd87dff67da82701,6726.66
5,2cc9089445046817a7539d90805e6e5a,6081.54
6,a96610ab360d42a2e5335a3998b4718a,4950.34
7,b4c4b76c642808cbe472a32b86cddc95,4809.44
8,199af31afc78c699f0dbf71fb178d4d4,4764.34
9,8dbc85d1447242f3b127dda390d56e19,4681.78


### 04. Quais são os últimos 10 pedidos com os menores valores?

In [54]:
query4 = """
select order_id,
       payment_value
from order_payments
where payment_value <> 0
order by payment_value asc
limit 10
"""

df4 = pd.read_sql_query(query4, con=conn)
df4

,order_id,payment_value
0,7db5f2eb8f5f54db9f9e71ba4296bcbf,0.01
1,fb4de3600d359f84927517e78ff9ba54,0.01
2,0218c7a4fb8d5b1bd22c82b783b8359c,0.01
3,ca4b9f3ce6fc19e8533501cf8c6b832e,0.01
4,636f0241ddc83a3b9e37a8088167bd45,0.01
5,25b5b0ea53b7d5a2d5712a0d9d0b3649,0.01
6,4884bd0f5624b0b791920965686cf6ff,0.03
7,bb1d699eccd1fe0e0b22aea230eab2d0,0.03
8,d8c4293616878e2f015351d8f1445880,0.05
9,2a4cf847d8e23123b4421ef7f7dc3d9f,0.05


### 05. Qual a média do valor de pagamento por tipo de pagamento?

In [28]:
query5 = """
select payment_type,
    avg(payment_value) as media_valor
from order_payments
group by payment_type
"""

df5 = pd.read_sql_query(query5, con=conn)
df5

,payment_type,media_valor
0,boleto,145.034435
1,credit_card,163.319021
2,debit_card,142.570170
3,not_defined,0.000000
4,voucher,65.703354


### 06. Quais os top 5 clientes com os maiores valores de pagamento no boleto?

In [57]:
query = """
SELECT * FROM orders
"""

table = pd.read_sql_query(query,conn)
table

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00
...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00


In [67]:
query6 = """
select customer_id,
       payment_type,
       sum(payment_value) as total_pedido
from orders o
left join order_payments op on (op.order_id = o.order_id)
where payment_type = 'boleto'
group by customer_id
order by total_pedido desc
limit 5
"""

df6 = pd.read_sql_query(query6, con=conn)
df6

,customer_id,payment_type,total_pedido
0,ec5b2ba62e574342386871631fafd3fc,boleto,7274.88
1,f48d464a0baaea338cb25f816991ab1f,boleto,6922.21
2,3fd6777bbce08a352fddd04e4a7cc8f6,boleto,6726.66
3,05455dfa7cd02f13d132aa7a6a9729c6,boleto,6081.54
4,31e83c01fce824d0ff786fcd48dad009,boleto,3979.55


### 07. Quais os top 5 clientes com os maiores valores de pagamento no cartão de crédito?

In [68]:
query7 = """
select customer_id,
       payment_type,
       sum(payment_value) as total_pedido
from orders o
left join order_payments op on (op.order_id = o.order_id)
where payment_type = 'credit_card'
group by customer_id
order by total_pedido desc
limit 5
"""

df7 = pd.read_sql_query(query7, con=conn)
df7

,customer_id,payment_type,total_pedido
0,1617b1357756262bfa56ab541c47bc16,credit_card,13664.08
1,c6e2731c5b391845f6800c97401a43a9,credit_card,6929.31
2,df55c14d1476a9a3467f131269c2477f,credit_card,4950.34
3,e0a2412720e9ea4f26c1ac985f6a7358,credit_card,4809.44
4,24bbf5fd2f2e1b359ee7de94defc4a15,credit_card,4764.34


### 08. Quais os 10 produtos mais caros?

In [48]:
query = """
SELECT * FROM items
"""

table = pd.read_sql_query(query,conn)
table

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72


In [52]:
query8 = """
select product_id, price
from items
group by product_id
order by price desc
limit 10
"""

df8 = pd.read_sql_query(query8, con=conn)
df8

,product_id,price
0,489ae2aa008f021502940f251d4cce7f,6735.00
1,69c590f7ffc7bf8db97190b6cb6ed62e,6729.00
2,1bdf5e6731585cf01aa8169c7028d6ad,6499.00
3,a6492cc69376c469ab6f61d8f44de961,4799.00
4,c3ed642d592594bb648ff4a04cee2747,4690.00
5,259037a6a41845e455183f89c5035f18,4590.00
6,a1beef8f3992dbd4cd8726796aa69c53,4399.87
7,6cdf8fc1d741c76586d8b6b15e9eef30,4099.99
8,6902c1962dd19d540807d0ab8fade5c6,3999.90
9,4ca7b91a31637bd24fb8e559d5e015e4,3999.00


### 09. Quais os 10 produtos mais baratos?

In [51]:
query9 = """
select product_id, price
from items
group by product_id
order by price asc
limit 10
"""

df9 = pd.read_sql_query(query9, con=conn)
df9

,product_id,price
0,8a3254bee785a526d548a81a9bc3c9be,0.85
1,05b515fdc76e888aada3c6d66c201dff,1.20
2,270516a3f41dc035aa87d220228f844c,1.20
3,46fce52cef5caa7cc225a5531c946c8b,2.20
4,310dc32058903b6416c71faff132df9e,2.29
5,680cc8535be7cc69544238c1d6a83fe8,2.90
6,2e8316b31db34314f393806fd7b6e185,2.99
7,44d53f1240d6332232e4393c06500475,3.00
8,29781581fb82fe2389560a3a5331d0ee,3.06
9,1716ea399ed8ee62ba811e6f55180f45,3.49


### 10. Quais as 10 categorias mais compradas?

In [70]:
query = """
SELECT * FROM items
"""

table = pd.read_sql_query(query,conn)
table

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72


In [46]:
query = """
SELECT * FROM products
"""

table = pd.read_sql_query(query,conn)
table

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0
...,...,...,...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,moveis_decoracao,45.0,67.0,2.0,12300.0,40.0,40.0,40.0
32947,bf4538d88321d0fd4412a93c974510e6,construcao_ferramentas_iluminacao,41.0,971.0,1.0,1700.0,16.0,19.0,16.0
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,cama_mesa_banho,50.0,799.0,1.0,1400.0,27.0,7.0,27.0
32949,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,60.0,156.0,2.0,700.0,31.0,13.0,20.0


In [86]:
query10 = """
select product_category_name,
       count(i.order_id) as qtd_vendas
from items i
left join products p on (p.product_id = i.product_id)
group by product_category_name
order by qtd_vendas desc
limit 10
"""

df10 = pd.read_sql_query(query10, con=conn)
df10

,product_category_name,qtd_vendas
0,cama_mesa_banho,11115
1,beleza_saude,9670
2,esporte_lazer,8641
3,moveis_decoracao,8334
4,informatica_acessorios,7827
5,utilidades_domesticas,6964
6,relogios_presentes,5991
7,telefonia,4545
8,ferramentas_jardim,4347
9,automotivo,4235


### 11. Quais os 5 produtos com maior número de reviews?

In [108]:
query = """
SELECT * FROM items
"""

table = pd.read_sql_query(query,conn)
table.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [107]:
query = """
SELECT * FROM order_reviews
"""

table = pd.read_sql_query(query,conn)
table.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,None,None,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,None,None,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,None,None,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,None,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,None,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [113]:
query11 = """
select i.product_id,
       count(review_id) as qtd_reviews
from order_reviews r
left join items i on (r.order_id = i.order_id)
where i.product_id <> 'None'
group by i.product_id
order by qtd_reviews desc
limit 5
"""

df11 = pd.read_sql_query(query11, con=conn)
df11

,product_id,qtd_reviews
0,aca2eb7d00ea1a7b8ebd4e68314663af,524
1,422879e10f46682990de24d770e7f83d,486
2,99a4788cb24856965c36a24e339b6058,482
3,389d119b48cf3043d311335e499d9c6b,391
4,368c6c730842d78016ad823897a372db,388


### 12. Quais os top 10 produtos sem nenhum review?

In [120]:
query12 = """
select i.product_id,
       count(review_id) as qtd_prod_sem_reviews
from order_reviews r
cross join items i on (r.order_id = i.order_id)
where i.product_id <> 'None' and review_comment_message is null
group by i.product_id
order by qtd_prod_sem_reviews desc
limit 10
"""

df12 = pd.read_sql_query(query12, con=conn)
df12

,product_id,qtd_prod_sem_reviews
0,aca2eb7d00ea1a7b8ebd4e68314663af,330
1,422879e10f46682990de24d770e7f83d,257
2,99a4788cb24856965c36a24e339b6058,245
3,53b36df67ebb7c41585e8d54d6772e08,206
4,389d119b48cf3043d311335e499d9c6b,197
5,d1c427060a0f73f6b889a5c7c61f2ac4,195
6,368c6c730842d78016ad823897a372db,179
7,53759a2ecddad2bb87a079a1f1519f73,177
8,154e7e31ebfa092203795c972e5804a6,175
9,3dd2a17168ec895c781a9191c1e95ad7,162


### 13. Quais os 10 clientes com maior quantidade de pedidos?

In [123]:
query = """
SELECT * FROM orders
"""

table = pd.read_sql_query(query,conn)
table.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [133]:
query13 = """
select c.customer_id,
       count(distinct o.order_id) as qtd_pedidos
from orders o
left join customer c on (c.customer_id = o.customer_id)
group by c.customer_id
order by qtd_pedidos desc
limit 10
"""

df13 = pd.read_sql_query(query13, con=conn)
df13

,customer_id,qtd_pedidos
0,ffffe8b65bbe3087b653a978c870db99,1
1,ffffa3172527f765de70084a7e53aae8,1
2,ffff42319e9b2d713724ae527742af25,1
3,fffeda5b6d849fbd39689bb92087f431,1
4,fffecc9f79fd8c764f843e9951b11341,1
5,fffcb937e9dd47a13f05ecb8290f4d3e,1
6,fffc22669ca576ae3f654ea64c8f36be,1
7,fffb97495f78be80e2759335275df2aa,1
8,fffa0238b217e18a8adeeda0669923a3,1
9,fff93c1da78dafaaa304ff032abc6205,1


### 14. Quais os 10 clientes com a menor quantidade de pedidos?

In [134]:
query14 = """
select c.customer_id,
       count(distinct o.order_id) as qtd_pedidos
from orders o
left join customer c on (c.customer_id = o.customer_id)
group by c.customer_id
order by qtd_pedidos asc
limit 10
"""

df14 = pd.read_sql_query(query14, con=conn)
df14

,customer_id,qtd_pedidos
0,00012a2ce6f8dcda20d059ce98491703,1
1,000161a058600d5901f007fab4c27140,1
2,0001fd6190edaaf884bcaf3d49edf079,1
3,0002414f95344307404f0ace7a26f1d5,1
4,000379cdec625522490c315e70c7a9fb,1
5,0004164d20a9e969af783496f3408652,1
6,000419c5494106c306a97b5635748086,1
7,00046a560d407e99b969756e0b10f282,1
8,00050bf6e01e69d5c0fd612f1bcfb69c,1
9,000598caf2ef4117407665ac33275130,1


### 15. Quais vendedores existem na base?

In [136]:
query = """
SELECT * FROM sellers
"""

table = pd.read_sql_query(query,conn)
table.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [138]:
query15 = """
select distinct seller_id
from sellers
"""

df15 = pd.read_sql_query(query15, con=conn)
df15

,seller_id
0,3442f8959a84dea7ee197c632cb2df15
1,d1b65fc7debc3361ea86b5f14c68d2e2
2,ce3ad9de960102d0677a81f5d0bb7b2d
3,c0f3eea2e14555b6faeea3dd58c1b1c3
4,51a04a8a6bdcb23deccc82b0b80742cf
...,...
3090,98dddbc4601dd4443ca174359b237166
3091,f8201cab383e484733266d1906e2fdfa
3092,74871d19219c7d518d0090283e03c137
3093,e603cf3fec55f8697c9059638d6c8eb5


### 16. Qual a distribuição de vendedores por estado?

In [141]:
query16 = """
select seller_state,
      count(seller_id) as qtd_vendedores
from sellers
group by seller_state
order by qtd_vendedores
"""

df16 = pd.read_sql_query(query16, con=conn)
df16

,seller_state,qtd_vendedores
0,AC,1
1,AM,1
2,MA,1
3,PA,1
4,PI,1
5,RO,2
6,SE,2
7,MT,4
8,MS,5
9,RN,5


### 17. Qual a distribuição de clientes por estado?

In [142]:
query = """
SELECT * FROM customer
"""

table = pd.read_sql_query(query,conn)
table.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [147]:
query17 = """
select customer_state,
      count(customer_id) as qtd_clientes
from customer
group by customer_state
order by qtd_clientes asc
"""

df17 = pd.read_sql_query(query17, con=conn)
df17

,customer_state,qtd_clientes
0,RR,46
1,AP,68
2,AC,81
3,AM,148
4,RO,253
5,TO,280
6,SE,350
7,AL,413
8,RN,485
9,PI,495


### 18. Quais são os top 10 vendedores que mais receberam pagamentos por boleto?

In [149]:
query = """
SELECT * FROM items
"""

table = pd.read_sql_query(query,conn)
table.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [151]:
query = """
SELECT * FROM order_payments
"""

table = pd.read_sql_query(query,conn)
table.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [154]:
query18 = """
select seller_id,
       payment_type,
       count(payment_type) as total_pedido
from items i
left join order_payments op on (op.order_id = i.order_id)
where payment_type = 'boleto'
group by seller_id
order by total_pedido desc
limit 10
"""

df18 = pd.read_sql_query(query18, con=conn)
df18

,seller_id,payment_type,total_pedido
0,1f50f920176fa81dab994f9023523100,boleto,541
1,6560211a19b47992c3666cc44a7e94c0,boleto,419
2,cc419e0650a3c5ba77189a1882b7556a,boleto,395
3,955fee9216a65b617aa5c0531780ce60,boleto,386
4,7c67e1448b00f6e969d365cea6b010ab,boleto,363
5,4a3ca9315b744ce9f8e9374361493884,boleto,338
6,da8622b14eb17ae2831f4ac5b9dab84a,boleto,329
7,1025f0e2d44d7041d6cf58b6550e0bfa,boleto,289
8,8b321bb669392f5163d04c59e235e066,boleto,255
9,7a67c85e85bb2ce8582c35f2203ad736,boleto,235


### 19. Quais são os 10 piores vendedores em termos de número de vendas?

In [159]:
query19 = """
select seller_id,
       count(distinct order_id) as total_pedido
from items 
group by seller_id
order by total_pedido asc
limit 10
"""

df19 = pd.read_sql_query(query19, con=conn)
df19

,seller_id,total_pedido
0,001e6ad469a905060d959994f1b41e4f,1
1,003554e2dce176b5555353e4f3555ac8,1
2,00ab3eff1b5192e5f1a63bcecfee11c8,1
3,00d8b143d12632bad99c0ad66ad52825,1
4,010da0602d7774602cd1b3f5fb7b709e,1
5,011b0eaba87386a2ae96a7d32bb531d1,1
6,028872bfa080090a9d0abd4f1af168f8,1
7,0336182e1b3e92f029d5354832045fdf,1
8,04843805947f0fc584fc1969b6e50fe7,1
9,04ee0ec01589969663ba5967c0e0bdc0,1


### 20. Quantos produtos são comprados, em média, por pedido?

In [163]:
query20 = """
with prod_por_pedido as(
        select order_id,
               count(product_id) as qtd_produtos
        from items 
        group by order_id
        order by qtd_produtos asc
)

select avg(qtd_produtos) as media_produtos
from prod_por_pedido
"""

df20 = pd.read_sql_query(query20, con=conn)
df20

,media_produtos
0,1.141731
